In [1]:
%%writefile customer_portal.py
import streamlit as st
from firebase_config import db
import datetime

def customer_portal():
    st.title("🚗 DVLA Spintex - Customer Portal")

    # Sidebar menu
    menu = ["Add Service Request", "View My Requests", "Book Appointment"]
    choice = st.sidebar.selectbox("Menu", menu)

    if choice == "Add Service Request":
        st.subheader("📄 New Vehicle Service Request")
        name = st.text_input("Full Name")
        email = st.text_input("Email")
        vehicle_reg = st.text_input("Vehicle Registration Number")
        service_type = st.selectbox("Service Type",
                                    ["Registration", "Renewal", "License Replacement"])

        if st.button("Submit Request"):
            request_id = f"req_{int(datetime.datetime.now().timestamp())}"
            db.collection("customer_requests").document(request_id).set({
                "name": name,
                "email": email,
                "vehicle_reg": vehicle_reg,
                "service_type": service_type,
                "status": "pending",
                "submitted_at": datetime.datetime.utcnow().isoformat()
            })
            st.success(f"✅ Request submitted! Your ID: {request_id}")

    elif choice == "View My Requests":
        st.subheader("📋 My Requests")
        email = st.text_input("Enter your email to view requests")
        if st.button("Search"):
            docs = db.collection("customer_requests").where("email", "==", email).stream()
            for doc in docs:
                data = doc.to_dict()
                st.write(f"**Request ID:** {doc.id}")
                st.write(f"Vehicle: {data['vehicle_reg']}")
                st.write(f"Service: {data['service_type']}")
                st.write(f"Status: {data['status']}")
                st.write("---")

    elif choice == "Book Appointment":
        st.subheader("📅 Book Appointment")
        name = st.text_input("Full Name")
        purpose = st.selectbox("Purpose", ["Registration", "Renewal", "License Replacement"])
        preferred_time = st.datetime_input("Preferred Date & Time")

        if st.button("Book Appointment"):
            appt_id = f"appt_{int(datetime.datetime.now().timestamp())}"
            db.collection("appointments").document(appt_id).set({
                "name": name,
                "purpose": purpose,
                "preferred_time": preferred_time.isoformat()
            })
            st.success(f"✅ Appointment booked! ID: {appt_id}")

# For running directly (debug mode)
if __name__ == "__main__":
    import streamlit.web.cli as stcli
    import sys
    sys.argv = ["streamlit", "run", __file__]
    sys.exit(stcli.main())


Writing customer_portal.py
